<a href="https://colab.research.google.com/drive/1DYU9uGaIN50-snUbAi6Z8DlPWpKrBxcA?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install the ucimlrepo package
%pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\alexa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
htru2 = fetch_ucirepo(id=372)

# data (as pandas dataframes)
X = htru2.data.features
y = htru2.data.targets

# metadata
print(htru2.metadata)

# variable information
print(htru2.variables)

{'uci_id': 372, 'name': 'HTRU2', 'repository_url': 'https://archive.ics.uci.edu/dataset/372/htru2', 'data_url': 'https://archive.ics.uci.edu/static/public/372/data.csv', 'abstract': 'Pulsar candidates collected during the HTRU survey. Pulsars are a type of star, of considerable scientific interest. Candidates must be classified in to pulsar and non-pulsar classes to aid discovery.', 'area': 'Physics and Chemistry', 'tasks': ['Classification', 'Clustering'], 'characteristics': ['Multivariate'], 'num_instances': 17898, 'num_features': 8, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2015, 'last_updated': 'Wed Apr 03 2024', 'dataset_doi': '10.24432/C5DK6R', 'creators': ['Robert Lyon'], 'intro_paper': {'ID': 460, 'type': 'NATIVE', 'title': 'Fifty years of pulsar candidate selection: from simple filters to a new principled real-time classification approach', 'a

In [3]:
#importando bibliotecas de treino, acurácia, precisão e recall
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

#função para calcular métricas médias apos as cinco execuções
def calculaMetrica(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acuracia = accuracy_score(y_test, y_pred)
    precisao = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    return acuracia, precisao, recall

In [4]:
#modelo KNN para armazenar exibir os resultados
modelos = {
    "KNN": KNeighborsClassifier()
}

# Variáveis para armazenar os resultados
resultados = {nomeModelo: {"acurácia": [], "precisão": [], "recall": []} for nomeModelo in modelos.keys()}

In [5]:
#utilização do 'for' para repetir 5 vezes a divisão do esquema de validação holdout e o treinamento para cada modelo
for _ in range(5):
  # Utilizando 6000 amostras para treino e teste selecionadas aleatóriamente
  X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), train_size=6000, random_state=None)

  #calculando as métricas para cada modelo
  for nomeModelo, modelo in modelos.items():
    acuracia, precisao, recall = calculaMetrica(modelo, X_train, X_test, y_train, y_test)
    resultados[nomeModelo]["acurácia"].append(acuracia)
    resultados[nomeModelo]["precisão"].append(precisao)
    resultados[nomeModelo]["recall"].append(recall)

#calculando as médias das métricas para cada algoritmo
averages = {nomeModelo: {metric: np.mean(scores) for metric, scores in metrics.items()} for nomeModelo, metrics in resultados.items()}
averages

{'KNN': {'acurácia': np.float64(0.9720121028744326),
  'precisão': np.float64(0.904387793325135),
  'recall': np.float64(0.776041056865248)}}

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import time

#carregando arquivos CSV HTRU
df = pd.read_csv('HTRU_2.csv', header=None)

In [7]:
#separação por rótulo
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

#escolhendo cinco valores diferentes/aleatórios para k
k_valores = [1, 2, 4, 6, 9]
validacaoResultados = {}

#dividindo a base de treinamento em duas partes iguais: 50% p treino e 50% p validação
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, train_size=6000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.5, random_state=42)

In [8]:
#avaliando cada valor de k
for k in k_valores:
  tempoInicio = time.time()

  #treinando e avaliando com o valor de k. KNN com métrica de distancia euclidiana
  knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
  knn.fit(X_train, y_train)

  #calculando a acurácia na validação
  validacaoAcuracia = knn.score(X_val, y_val)

  #Para armazenar resultados e tempo
  validacaoResultados[k] = {
    "Acurácia:": validacaoAcuracia,
    "Tempo de execução": time.time() - tempoInicio,
    #(letra b) num de protótipos usados para cada k
    "num. prototipos": len(X_train)
  }

print(validacaoResultados)

{1: {'Acurácia:': 0.967, 'Tempo de execução': 0.10806512832641602, 'num. prototipos': 3000}, 2: {'Acurácia:': 0.973, 'Tempo de execução': 0.11349248886108398, 'num. prototipos': 3000}, 4: {'Acurácia:': 0.974, 'Tempo de execução': 0.19026660919189453, 'num. prototipos': 3000}, 6: {'Acurácia:': 0.9746666666666667, 'Tempo de execução': 0.10392570495605469, 'num. prototipos': 3000}, 9: {'Acurácia:': 0.9743333333333334, 'Tempo de execução': 0.10547780990600586, 'num. prototipos': 3000}}


In [9]:
#Verificando a proporção das classes
contasClasses = y.value_counts(normalize=True)
print("Proporção das classes:")
print(contasClasses)

Proporção das classes:
8
0    0.908426
1    0.091574
Name: proportion, dtype: float64
